In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')


import re
import os, sys

import numpy as np # linear algebra

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

import shutil

from config import SEED, PARTIAL_TRAIN, TEST_SIZE, NUM_LABELS 
from config import MAX_SEQUENCE_LENGTH, NUM_EPOCH, LEARNING_RATE, BATCH_SIZE
from config import ACCUMULATION_STEPS, INPUT_DIR, WORK_DIR, TOXICITY_COLUMN, DATA_DIR
from config import BERT_MODEL_NAME, FINE_TUNED_MODEL_PATH, BERT_MODEL_PATH

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

from utils import set_seed, convert_lines_onfly, preprocess

device = torch.device('cpu')

In [3]:
set_seed(SEED)

In [6]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /Users/matthewvu/Desktop/ToxicDetection-master/input/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm

'../working/bert_config.json'

In [7]:
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/uncased_L-12_H-768_A-12/'+'bert_config.json')

In [8]:
## instantiate bert pretrained model and tokenizer
model = BertForSequenceClassification(bert_config, num_labels=NUM_LABELS)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=True)

## load saved model
model.load_state_dict(torch.load(FINE_TUNED_MODEL_PATH,map_location=torch.device('cpu'))) 
for p in model.parameters():
    p.requires_grad = False

## use gpu
##model.cuda()

def predict_toxicity(sentence: str) -> float :
    """
    predict the toxicity level from a sentence
    """    

    model.eval()
    
    X = np.array([str(sentence)])
    test_preds = torch.zeros((len(X)))
    
    Xp = convert_lines_onfly(X, MAX_SEQUENCE_LENGTH, tokenizer)
    y_pred = model(torch.from_numpy(Xp).to(device))
    test_preds[0] += torch.sigmoid(y_pred[0,0].cpu())

    return float(test_preds[0])

In [9]:
sentence = "Oh sh*t!! What an awesome goal, I nearly missed it…"
#sentence = "Yet call out all Muslims for the acts of a few will get you pilloried.   So why is it okay to smear an entire religion over these few idiots?  Or is this because it's okay to bash Christian sects?"
# sentence = "Sorry to have to do this, but just to see if profanity filtering is enabled"
predict_toxicity(sentence)

tensor([0.6917])


0.6917122602462769